## Titanic

### 이해한 내용
- 우선 빠른 시간 내에 이해를 필요로 해서 한글본으로 보았습니다.
- 가장 먼저 결측치를 확인해보았는데 결측치를 경우에 따라 제거하고 채우는 방식을 확인할 수 있었습니다.
    - 결측값이 너무 많은 경우(cabin) : 컬럼을 아예 제거
    - 결측값을 중앙값으로 채우거나 최빈값으로 채우는 방법
- SibSp와 Parch 변수들은 모두 가족을 의미하는 것이기 때문에 다중 공선성을 고려하여 하나로 합친 다음에 여행을 혼자 왔는지 아닌지로 파생변수를 만들었습니다.
오히려 가족이 1~2명 있는 것보다 혼자 온 경우 많이 죽었기 때문에 이렇게 혼자 왔는지 아닌지로 나누는 방법도 좋다고 생각합니다.

목차:

**[I. 데이터 불러오기 및 확인](#one)**

- [a. 데이터 불러오기 및 결측치 확인](#one-a)

**[II. EDA & FE](#two)**

- [a. Pclass](#two-a)
- [b. Sex](#two-b)
- [c. SibSp](#two-c)
- [d. Parch](#two-d)
- [e. Name](#two-e)
- [f. Age](#two-f)
- [g. Embarked](#two-g)
- [h. Fare](#two-h)
- [i. cabin](#two-i)

    
**[III. 모델링](#three)**

- [a. 로지스틱 회귀](#three-a)
- [b. kfold](#three-b)
- [c. 파라미터 튜닝 & GridSearchCV](#three-c)

**[IV. 예측](#four)**

- [a. 앙상블](#four-a)
- [b. voting](#four-b)
- [c. stacking](#four-c)

## I. 데이터 불러오기 및 확인 <a id="one"></a>

1. 변수 | 설명
------- | -------
**PassengerId** | id
**Survived** | 생존 여부 (0 = 사망, 1 = 생존) 
**Pclass** | 티켓 클래스 (1 = 1등석, 2 = 2등석, 3 = 3등석)
**name** | 이름  
**Sex** | 성별
**Age** | 나이
**SibSp** | 함께 탑승한 자녀 , 배우자 의 수
**Parch** | 함께 탑승한 부모님 , 아이들 의 수
**Fare** | 요금
**Ticket** | 티켓 번호
**Cabin** | 수하물 번호
**Embarked** | 선착장 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
plt.rcParams["font.family"] = 'Malgun Gothic'

### a. 데이터 불러오기 및 결측치 확인 <a id="one-a"></a>

In [ ]:
train = pd.read_csv("../input/titanic/train.csv")
test = pd.read_csv("../input/titanic/test.csv")
sample_submission = pd.read_csv('../input/titanic/gender_submission.csv')

In [ ]:
train.head(3)

In [ ]:
test.head()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

> 결측치 확인 결과 AGE와 Cabin, Embarked 피처에 대해 처리를 해줘야 될거 같다

## II. EDA & FE <a id="two"></a> 

In [ ]:
fe_name = list(test)
df_train = train[fe_name]
df = pd.concat((df_train,test))

In [ ]:
print(train.shape, test.shape, df.shape)

In [ ]:
target = train['Survived']

In [ ]:
def stack_plot(feature):
    survived = train[train['Survived'] == 1][feature].value_counts()
    dead = train[train['Survived'] == 0][feature].value_counts()
    df = pd.DataFrame([survived,dead])
    df.index = ['survived','dead']
    df.plot(kind='bar', stacked= True, figsize = (10,5))

In [ ]:
lable = ['survived','dead']
plt.title('생존 수')
plt.pie(train['Survived'].value_counts(),labels= lable,autopct='%.f%%')

>전체 62%가 사망한것을 볼 수 있다.

### a.Pclass <a id="two-a"></a>

In [ ]:
stack_plot('Pclass')

In [ ]:
Pclass_encoded = pd.get_dummies(df['Pclass'],prefix= 'Pclass')
df = pd.concat((df,Pclass_encoded), axis=1)
df = df.drop(columns = 'Pclass')

>3등석 사람이 많이 죽은 것을 볼 수 있으며 Pclass 피처는 카테코리 변수로 판단하여 원핫 인코딩을 위해 더미변수로 만듭니다.

### b.Sex <a id="two-b"></a>

In [ ]:
stack_plot('Sex')

In [ ]:
sex_encoded = pd.get_dummies(df['Sex'],prefix= 'Sex')
df = pd.concat((df,sex_encoded), axis=1)
df = df.drop(columns = 'Sex')

In [ ]:
df.drop('Sex_female', axis=1, inplace=True)

>Sex 피처는 카테코리 변수로 판단하여 원핫 인코딩을 위해 더미변수로 만듭니다.

### c.SibSp <a id="two-c"></a>

In [ ]:
stack_plot('SibSp')

### d.Parch <a id="two-d"></a>

In [ ]:
stack_plot('Parch')

In [ ]:
df['Travelpeople']=df["SibSp"]+df["Parch"]
df['TravelAlone']=np.where(df['Travelpeople']>0, 0, 1)

In [ ]:
df.drop('SibSp', axis=1, inplace=True)
df.drop('Parch', axis=1, inplace=True)

In [ ]:
df.drop('Travelpeople', axis=1, inplace=True)

>SibSp와 Parch 변수들은 같이 여행을 온 경우로 판단하여
다중 공선성을 고려하여 하나로 합친 다음에 여행을 혼자 왔는지 아닌지로 파생변수를 만듭니다.

### e.Name <a id="two-e"></a>

>Mr - 남자
Mrs. 결혼한 기혼 여자
Miss. 결혼하지 않은 미혼 여자
Ms. 여자일경우

In [ ]:
df['New_name']  = df['Name'].str.extract('([A-Za-z]+)\.', expand=False)
train['New_name']  = train['Name'].str.extract('([A-Za-z]+)\.', expand=False)

In [ ]:
train ['New_name'] =  train['New_name'].map({"Mr": 0 , "Mrs":2, "Miss":1,"Dr":3,"Rev":3,
                                             "Mlle":3,"Major":3,"Col":3,"Ms":3,"Jonkheer":3,
                                             "Sir" :3,"Lady":3,"Mme":3,"Capt":3,"Don":3,"Countess":3})
df['New_name'] =  df['New_name'].map({"Mr": 0 , "Mrs":2, "Miss":1,"Dr":3,"Rev":3,
                                             "Mlle":3,"Major":3,"Col":3,"Ms":3,"Jonkheer":3,
                                             "Sir" :3,"Lady":3,"Mme":3,"Capt":3,"Don":3,"Countess":3})

In [ ]:
stack_plot('New_name')

In [ ]:
df['New_name'] = df['New_name'].fillna('0')

In [ ]:
df = df.astype({'New_name':'float'})

In [ ]:
df = df.drop(columns = 'Name')

In [ ]:
New_name_encoded = pd.get_dummies(df['New_name'],prefix= 'New_name')
df = pd.concat((df,New_name_encoded), axis=1)
df = df.drop(columns = 'New_name')

> 이름의 경우 정규 표현식을 이용해서 Name 피처를 처리해주고 남자 여자 그외로 매핑해준다.
Mr - 남자 / Mrs. 결혼한 기혼 여자 / Miss. 결혼하지 않은 미혼 여자 / MS. 여자일 경우
그래프를 보면 남자가 많이 사망한것으로 보며 결측치를 남자로 채웠고 더미변수를 만들어 주었다.

### f.Age <a id="two-f"></a>

In [ ]:
df['Age'].hist(bins = 15)

In [ ]:
df['Age'].fillna(28, inplace = True)

>Age 피처의 데이터 확인결과 우측으로 기울어져있기 때문에 평균값을 사용하는 것보다 중앙값을 사용하기로 했다

### g.Embarked <a id="two-g"></a>

In [ ]:
sns.countplot(x= 'Embarked', data= df)

In [ ]:
df['Embarked'].fillna('S',inplace=True)

In [ ]:
Embarked_encoded = pd.get_dummies(df['Embarked'],prefix= 'Embarked')
df = pd.concat((df,Embarked_encoded), axis=1)
df = df.drop(columns = 'Embarked')

>Embarked 피처는 countplot 결과 S선착장이 많은 것을 볼 수 잇으며 결측치를 S로 대체하였고
카테코리 변수로 판단하여 원핫 인코딩을 위해 더미변수로 만듭니다.

### h.Fare <a id="two-h"></a>

In [ ]:
from scipy.stats import norm

In [ ]:
sns.distplot(train['Fare'],fit = norm)

>정규규분포가 아닌 매우 비대칭 분포 멱변환을 통해서 정규화를 시켜준다

In [ ]:
df['Fare'] = df['Fare'].map(lambda i : np.log(i) if i >0 else 0)

In [ ]:
sns.distplot(df['Fare'],fit = norm)

>정규규분포가 아닌 매우 비대칭 분포 멱변환을 통해서 정규화를 시켜준다

### i.Cabin <a id="two-i"></a>

In [ ]:
df['Cabin'].value_counts()

In [ ]:
df = df.drop(columns = 'Cabin')

>변수에 결측값이 너무 많아. 이 값은 승객의 클래스 및 지불 된 운임과 관련이있는 것으로 보고 드롭했습니다

In [ ]:
df = df.drop(['PassengerId','Ticket'],axis = 1)

> PassengerId , Ticket 피처들도 모델링 과정에 필요없는 변수들이므로 드롭해줍니다

In [ ]:
df.isnull().sum()

## III. 모델링 <a id="three"></a>

In [ ]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

### a. 로지스틱 회귀 <a id="three-a"></a>

In [ ]:
new_train = df[:train.shape[0]]
new_test = df[train.shape[0]:]

In [ ]:
cols=["Age", "Fare", "TravelAlone", "Pclass_1", "Pclass_2","Embarked_C","Embarked_S",
      "Sex_male",'New_name_0.0','New_name_1.0', 'New_name_2.0'] 

In [ ]:
X = new_train[cols]
Y = train['Survived']

In [ ]:
import statsmodels.api as sm
from scipy import stats
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)
logit_model=sm.Logit(Y,X)
result=logit_model.fit()
print(result.summary())

> 로지스틱 모델 확인 결과 Fare, Embarked_S,Embarked_C,New_name_0 피처에 대해서 p-value가 0.05 기준으로 높게 나와있으므로 해당 피처를 모두 제고하고 다시 확인해 봅니다.

In [ ]:
cols2 = ["Age", "TravelAlone", "Pclass_1", "Pclass_2","Embarked_S",
      "Sex_male",'New_name_1.0', 'New_name_2.0'] 

In [ ]:
X2=new_train[cols2]
Y=train['Survived']

logit_model=sm.Logit(Y,X2)
result=logit_model.fit()
print(result.summary())

> 로지스틱 모델 확인 결과 TravelAlone 피처에 대해서 p-value가 0.05 기준으로 높게 나와있으므로 피처를 모두 제고하고 다시 확인해 봅니다.

In [ ]:
cols3=["Age", "Pclass_1", "Pclass_2","Embarked_S",
      "Sex_male",'New_name_1.0', 'New_name_2.0'] 

In [ ]:
X3=new_train[cols3]
Y=train['Survived']

logit_model=sm.Logit(Y,X3)
result=logit_model.fit()
print(result.summary())

> 모든 피처들이 유의수준 0.05보다 낮은것을 확인 할 수 있습니다.

In [ ]:
f_test = new_test[cols3]

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(X3, Y)

print("모델 Accuracy : {:.2f}%".format(logreg.score(X3, Y)*100))

> 모델의 Accuracy를 높이기 위해서 로지스틱 회귀를 사용할 수 있겠지만 모델의 성능을 높이기 위해서
여러 모델을 활용하여 앙상블을 합니다

### b. kfold <a id="three-b"></a>

In [ ]:
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold

In [ ]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [ ]:
kfold = StratifiedKFold(n_splits=8)

In [ ]:
random_state = 1
clf = []

clf.append(XGBClassifier(random_state = random_state))
clf.append(LGBMClassifier(random_state = random_state))
clf.append(KNeighborsClassifier())
clf.append(RandomForestClassifier(random_state=random_state))
clf.append(GradientBoostingClassifier(random_state=random_state))
clf.append(DecisionTreeClassifier(random_state=random_state))
clf.append(LogisticRegression(random_state = random_state))
clf.append(SVC(random_state=random_state))

In [ ]:
clf_results = []
for classifier in clf :
    clf_results.append(cross_val_score(classifier, new_train, y = Y, scoring = "accuracy", cv = kfold, n_jobs=4))

In [ ]:
clf_means = []
clf_std = []
for clf_result in clf_results:
    clf_means.append(clf_result.mean())
    clf_std.append(clf_result.std())

In [ ]:
clf_re = pd.DataFrame({"CrossValMeans":clf_means,"CrossValerrors": clf_std})
clf_re

> 데이터 세트를 8개의 그룹으로 분할한 다음에 8개의 모델에 대해서 기본 모델을 구축한 뒤 gridsearchcv를 통해서 변수들을 최적화 시켜줍니다. kfold 확인 결과 4개의 모델을 이용해서 앙상블 하려고 합니다 

### c. 파라미터 튜닝 & GridSearchCV <a id="three-c"></a>

In [ ]:
# XGBoost 파라미터 튜닝 
XGB = XGBClassifier()
xgb_param_grid = {'learning_rate': [1,0.1,0.01,0.001],
              'n_estimators': [50, 100, 200, 500, 1000],
              'max_depth' : [1,3,5,10,50]}
gsXGB = GridSearchCV(XGB,param_grid = xgb_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)
gsXGB.fit(new_train,Y)
XGB_best = gsXGB.best_estimator_

# 최고 점수
gsXGB.best_score_

In [ ]:
#LGBMClassifier 파라미터 튜닝
LGB = LGBMClassifier()
lgb_param_grid = {
    'n_estimators': [400, 700, 1000], 
    'max_depth': [15,20,25],
    'num_leaves': [50, 100, 200],
    'min_split_gain': [0.3, 0.4],
}
gsLGB = GridSearchCV(LGB,param_grid = lgb_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)
gsLGB.fit(new_train,Y)
LGB_best = gsLGB.best_estimator_

# 최고 점수
gsLGB.best_score_

In [ ]:
# RandomForestClassifier 파라미터 튜닝 
RFC = RandomForestClassifier()

rf_param_grid = {"max_depth": [None],
              "max_features": [1, 3, 7],
              "min_samples_split": [2, 3, 7],
              "min_samples_leaf": [1, 3, 7],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}
gsRFC = GridSearchCV(RFC,param_grid = rf_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)
gsRFC.fit(new_train,Y)
RFC_best = gsRFC.best_estimator_

# 최고 점수
gsRFC.best_score_

>Extra Tree Classifier와 유사하게 Random Forest는 여러 의사 결정 트리 분류자를 데이터 집합의 다양한 하위 샘플에 맞추고 평균을 사용하여 예측 정확도를 개선하고 과적 합을 제어합니다. 하위 샘플 크기는 항상 원래 입력 샘플 크기와 동일하지만 bootstrap = True (기본값) 인 경우 샘플이 대체로 그려집니다.

>ET와 RF의 차이점-
>1) 분할에서 변수를 선택할 때, 훈련 세트의 부트 스트랩 샘플 대신 전체 훈련 세트에서 샘플이 추출됩니다.
>2) 분할은 각 분할에서 샘플의 값 범위에서 무작위로 완전히 선택됩니다.

In [ ]:
# Gradient boosting 파라미터 튜닝
GBC = GradientBoostingClassifier()
gb_param_grid = {'loss' : ["deviance"],
              'n_estimators' : [100,200,300],
              'learning_rate': [0.1, 0.05, 0.01],
              'max_depth': [4, 8],
              'min_samples_leaf': [100,150],
              'max_features': [0.3, 0.1] 
              }
gsGBC = GridSearchCV(GBC,param_grid = gb_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)
gsGBC.fit(new_train,Y)
GBC_best = gsGBC.best_estimator_

# 최고 점수
gsGBC.best_score_

> 그라디언트 부스팅은 예측 모델을 작성하는 가장 강력한 기술 중 하나로 부스팅은 단계별 모델 방식으로 가산 모델을 작성하여 약한 학습자를 강력한 학습자로 변환하는 방법 
부스팅에서 각각의 새 트리는 원래 데이터 세트의 수정 된 버전에 적합하다.

## IV. 예측 <a id="four"></a>

### a.앙상블 <a id="four-a"></a>

In [ ]:
test_Survived_XGB = pd.Series(XGB_best.predict(new_test), name="XGB")
test_Survived_LGB = pd.Series(LGB_best.predict(new_test), name="LGB")
test_Survived_RFC = pd.Series(RFC_best.predict(new_test), name="RFC")
test_Survived_GBC = pd.Series(GBC_best.predict(new_test), name="GBC")

ensemble_results = pd.concat([test_Survived_XGB,test_Survived_LGB,
                              test_Survived_RFC,test_Survived_GBC],axis=1)
g= sns.heatmap(ensemble_results.corr(),annot=True)

> 4가지 모델을 heatmap을 통해서 상관관계를 비교해보고 상관성이 높은 RFC와 LGB 모델을 이용하여
앙상블 모델을 만들어서 결과를 제출합니다

### b.보팅 <a id="four-b"></a>

In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
votingC = VotingClassifier(estimators=[('XGB', XGB_best), ('LGB', LGB_best),
('RFC', RFC_best), ('GBC',GBC_best)], voting='soft', n_jobs=4)
votingC = votingC.fit(new_train, Y)  

In [ ]:
test_Survived = pd.Series(votingC.predict(new_test), name="Survived")

In [ ]:
submission = pd.DataFrame({
    "PassengerId" :test["PassengerId"],
    "Survived": test_Survived
})

In [ ]:
submission.to_csv('voting_titanic.csv',index=False) 

> 투표는 여러 머신 러닝 모델의 예측을 결합하는 가장 간단한 방법 중 하나입니다. 실제 분류기는 아니지만 각 알고리즘의 다른 특성을 이용하기 위해 병렬로 학습되고 평가되는 다른 세트에 대한 래퍼입니다. 투표를 기반으로 구축하고 예측 한 모델의 예측을 결합합니다.

### c.스태킹 <a id="four-c"></a>

In [ ]:
from mlxtend.classifier import StackingClassifier
from sklearn.utils.testing import ignore_warnings

In [ ]:
clf1 = XGB_best
clf2 = LGB_best
clf3 = RFC_best
clf4 = GBC_best

lr = LogisticRegression()
st_clf = StackingClassifier(classifiers=[clf1, clf1, clf2, clf3, clf4], meta_classifier=lr)
params = {'meta_classifier__C': [0.1,1.0,5.0,10.0] ,
          #'use_probas': [True] ,
          #'average_probas': [True] ,
          'use_features_in_secondary' : [True, False]
         }
with ignore_warnings(category=DeprecationWarning):
    st_clf_grid = GridSearchCV(estimator=st_clf, param_grid=params, cv=5, refit=True)
    st_clf_grid.fit(new_train, Y)
    st_clf_grid.best_score_

In [ ]:
with ignore_warnings(category=DeprecationWarning):    
    pred_all_stack = st_clf_grid.predict(new_test)

submission1 = pd.DataFrame({
    "PassengerId" :test["PassengerId"],
    "Survived": pred_all_stack
})
#submission1.to_csv('stack_clf.csv',index=False)

>Boosting, Stacking 및 Voting과 같은 분류에 앙상블 모델을 적용하여 타이타닉 생존자 예측을 하였다